book language를 최빈값과 isbn기반 추론으로 했을 때 어떤 차이가 있는가 비교


In [1]:
"""
v1 (최빈값) vs v2 (ISBN 기반) Language 분포 차이 분석
원래 결측이었던 데이터만 비교
"""

import pandas as pd
import numpy as np
from scipy import stats

print("=" * 80)
print("Language 추론 방법 비교: v1 (최빈값) vs v2 (ISBN 기반)")
print("=" * 80)

# 1. 원본 데이터로 결측치 위치 파악
print("\n[Step 1] 원본 데이터에서 결측치 파악")
books_original = pd.read_csv("/data/ephemeral/home/sojin/data/books.csv")
original_missing_mask = books_original["language"].isna()
original_missing_count = original_missing_mask.sum()
print(f"원본 Language 결측치: {original_missing_count:,}건")

# 원래 결측이었던 ISBN 리스트
original_missing_isbns = set(books_original[original_missing_mask]["isbn"].astype(str))

# 2. v1, v2 데이터 로드
print("\n[Step 2] v1, v2 데이터 로드")
books_v1 = pd.read_csv("/data/ephemeral/home/sojin/data/v1/books.csv")
books_v2 = pd.read_csv("/data/ephemeral/home/sojin/data/v2/books.csv")

# ISBN을 문자열로 통일
books_v1["isbn"] = books_v1["isbn"].astype(str)
books_v2["isbn"] = books_v2["isbn"].astype(str)

print(f"v1 총 데이터: {len(books_v1):,}건")
print(f"v2 총 데이터: {len(books_v2):,}건")

# 3. 원래 결측이었던 데이터만 필터링
print("\n[Step 3] 원래 결측이었던 데이터만 추출")
v1_filled = books_v1[books_v1["isbn"].isin(original_missing_isbns)].copy()
v2_filled = books_v2[books_v2["isbn"].isin(original_missing_isbns)].copy()

print(f"v1에서 추출: {len(v1_filled):,}건")
print(f"v2에서 추출: {len(v2_filled):,}건")

# 4. v1 Language 분포 (최빈값 기반)
print("\n" + "=" * 80)
print("📊 v1 (최빈값 기반) Language 분포")
print("=" * 80)
v1_lang_dist = v1_filled["language"].value_counts()
v1_lang_pct = (v1_lang_dist / len(v1_filled) * 100).round(2)

v1_dist_df = pd.DataFrame(
    {
        "언어": v1_lang_dist.index,
        "건수": v1_lang_dist.values,
        "비율(%)": v1_lang_pct.values,
    }
)

display(v1_dist_df)
print(f"\n총 언어 종류: {len(v1_lang_dist)}개")

# 5. v2 Language 분포 (ISBN 기반)
print("\n" + "=" * 80)
print("📊 v2 (ISBN 기반) Language 분포")
print("=" * 80)
v2_lang_dist = v2_filled["language"].value_counts()
v2_lang_pct = (v2_lang_dist / len(v2_filled) * 100).round(2)

v2_dist_df = pd.DataFrame(
    {
        "언어": v2_lang_dist.index,
        "건수": v2_lang_dist.values,
        "비율(%)": v2_lang_pct.values,
    }
)

display(v2_dist_df.head(20))
print(f"\n총 언어 종류: {len(v2_lang_dist)}개")

# 6. 직접 비교 (병합)
print("\n" + "=" * 80)
print("📊 v1 vs v2 언어 분포 직접 비교")
print("=" * 80)

# 모든 언어 리스트
all_languages = sorted(set(v1_lang_dist.index) | set(v2_lang_dist.index))

comparison_data = []
for lang in all_languages:
    v1_count = v1_lang_dist.get(lang, 0)
    v2_count = v2_lang_dist.get(lang, 0)
    v1_pct = (v1_count / len(v1_filled) * 100) if v1_count > 0 else 0
    v2_pct = (v2_count / len(v2_filled) * 100) if v2_count > 0 else 0
    diff = v2_count - v1_count

    comparison_data.append(
        {
            "언어": lang,
            "v1_건수": v1_count,
            "v1_%": round(v1_pct, 2),
            "v2_건수": v2_count,
            "v2_%": round(v2_pct, 2),
            "차이": diff,
            "차이_%": round(v2_pct - v1_pct, 2),
        }
    )

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values("v2_건수", ascending=False)

display(comparison_df.head(20))

# 7. 통계적 유의성 검정 (Chi-square test)
print("\n" + "=" * 80)
print("📈 통계적 유의성 검정 (Chi-square Test)")
print("=" * 80)

# v1과 v2의 언어 분포를 2x언어수 contingency table로 구성
contingency_data = []
for lang in all_languages:
    v1_count = v1_lang_dist.get(lang, 0)
    v2_count = v2_lang_dist.get(lang, 0)
    contingency_data.append([v1_count, v2_count])

contingency_table = np.array(contingency_data).T

# Chi-square test
chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)

print(f"Chi-square 통계량: {chi2:.2f}")
print(f"자유도 (degrees of freedom): {dof}")
print(f"p-value: {p_value:.10f}")
print(f"\n해석:")
if p_value < 0.001:
    print("  ✅ p < 0.001 → 매우 유의미한 차이 (***)")
    print("  → v1과 v2의 언어 분포는 통계적으로 명백히 다릅니다!")
elif p_value < 0.01:
    print("  ✅ p < 0.01 → 유의미한 차이 (**)")
    print("  → v1과 v2의 언어 분포는 통계적으로 다릅니다!")
elif p_value < 0.05:
    print("  ✅ p < 0.05 → 유의미한 차이 (*)")
    print("  → v1과 v2의 언어 분포는 통계적으로 다릅니다.")
else:
    print("  ❌ p >= 0.05 → 유의미한 차이 없음")
    print("  → v1과 v2의 언어 분포는 통계적으로 비슷합니다.")

# 8. 주요 차이점 요약
print("\n" + "=" * 80)
print("🔍 주요 차이점 요약")
print("=" * 80)

# v2에서 크게 증가한 언어 (v1 대비)
increased = (
    comparison_df[comparison_df["차이"] > 0]
    .sort_values("차이", ascending=False)
    .head(10)
)
if len(increased) > 0:
    print("\n[v2에서 증가한 언어 Top 10]")
    display(increased[["언어", "v1_건수", "v2_건수", "차이", "차이_%"]])

# v2에서 크게 감소한 언어 (v1 대비)
decreased = comparison_df[comparison_df["차이"] < 0].sort_values("차이").head(10)
if len(decreased) > 0:
    print("\n[v2에서 감소한 언어 Top 10]")
    display(decreased[["언어", "v1_건수", "v2_건수", "차이", "차이_%"]])

# 9. 다양성 지표
print("\n" + "=" * 80)
print("📊 언어 다양성 지표")
print("=" * 80)


# Shannon Entropy (정보 엔트로피)
def shannon_entropy(distribution):
    """Shannon Entropy 계산"""
    probabilities = distribution / distribution.sum()
    probabilities = probabilities[probabilities > 0]  # 0 제거
    return -np.sum(probabilities * np.log2(probabilities))


v1_entropy = shannon_entropy(v1_lang_dist.values)
v2_entropy = shannon_entropy(v2_lang_dist.values)


# Gini-Simpson Index (다양성 지수)
def gini_simpson(distribution):
    """Gini-Simpson diversity index"""
    probabilities = distribution / distribution.sum()
    return 1 - np.sum(probabilities**2)


v1_diversity = gini_simpson(v1_lang_dist.values)
v2_diversity = gini_simpson(v2_lang_dist.values)

diversity_comparison = pd.DataFrame(
    {
        "지표": ["고유 언어 수", "Shannon Entropy", "Gini-Simpson Index"],
        "v1 (최빈값)": [len(v1_lang_dist), f"{v1_entropy:.4f}", f"{v1_diversity:.4f}"],
        "v2 (ISBN)": [len(v2_lang_dist), f"{v2_entropy:.4f}", f"{v2_diversity:.4f}"],
        "해석": [
            "언어 종류가 많을수록 다양함",
            "높을수록 다양함 (max: log2(N))",
            "높을수록 다양함 (0~1)",
        ],
    }
)

display(diversity_comparison)

print("\n해석:")
if v2_entropy > v1_entropy:
    print(f"  ✅ v2가 v1보다 {v2_entropy - v1_entropy:.4f} 더 다양함 (Entropy)")
else:
    print(f"  ❌ v1이 v2보다 {v1_entropy - v2_entropy:.4f} 더 다양함 (Entropy)")

if v2_diversity > v1_diversity:
    print(
        f"  ✅ v2가 v1보다 {(v2_diversity - v1_diversity)*100:.2f}% 더 다양함 (Gini-Simpson)"
    )
else:
    print(
        f"  ❌ v1이 v2보다 {(v1_diversity - v2_diversity)*100:.2f}% 더 다양함 (Gini-Simpson)"
    )

# 10. 최종 결론
print("\n" + "=" * 80)
print("📋 최종 결론")
print("=" * 80)

summary = pd.DataFrame(
    {
        "항목": [
            "처리 방법",
            "총 처리 건수",
            "고유 언어 수",
            "최다 언어",
            "최다 언어 비율",
            "Shannon Entropy",
            "Gini-Simpson Index",
            "통계적 차이",
        ],
        "v1": [
            "최빈값 대체",
            f"{len(v1_filled):,}건",
            f"{len(v1_lang_dist)}개",
            f"{v1_lang_dist.index[0]}",
            f"{v1_lang_pct.iloc[0]:.1f}%",
            f"{v1_entropy:.4f}",
            f"{v1_diversity:.4f}",
            "-",
        ],
        "v2": [
            "ISBN 기반 추론",
            f"{len(v2_filled):,}건",
            f"{len(v2_lang_dist)}개",
            f"{v2_lang_dist.index[0]}",
            f"{v2_lang_pct.iloc[0]:.1f}%",
            f"{v2_entropy:.4f}",
            f"{v2_diversity:.4f}",
            f"p={p_value:.2e}",
        ],
    }
)

display(summary)

print("\n결론:")
if p_value < 0.001:
    print("  ✅ v1과 v2의 언어 분포는 통계적으로 매우 유의미하게 다릅니다!")
    print("  ✅ ISBN 기반 추론이 더 다양한 언어를 올바르게 식별했습니다.")
    print("  → v2 (ISBN 기반) 추론이 더 정확하고 의미있는 결과입니다.")
else:
    print("  ❌ v1과 v2의 언어 분포는 통계적으로 유의미한 차이가 없습니다.")
    print("  → 어느 방법을 사용해도 비슷한 결과입니다.")

print("\n" + "=" * 80)
print("분석 완료!")
print("=" * 80)

Language 추론 방법 비교: v1 (최빈값) vs v2 (ISBN 기반)

[Step 1] 원본 데이터에서 결측치 파악
원본 Language 결측치: 67,227건

[Step 2] v1, v2 데이터 로드
v1 총 데이터: 149,570건
v2 총 데이터: 149,570건

[Step 3] 원래 결측이었던 데이터만 추출
v1에서 추출: 67,227건
v2에서 추출: 67,227건

📊 v1 (최빈값 기반) Language 분포


,언어,건수,비율(%)
0,en,67227,100.0



총 언어 종류: 1개

📊 v2 (ISBN 기반) Language 분포


,언어,건수,비율(%)
0,en,56369,83.85
1,de,5284,7.86
2,fr,2420,3.60
3,es,2415,3.59
4,it,355,0.53
5,nl,125,0.19
6,pt,110,0.16
7,ja,38,0.06
8,ru,27,0.04
9,pl,24,0.04



총 언어 종류: 19개

📊 v1 vs v2 언어 분포 직접 비교


...


📊 언어 다양성 지표


,지표,v1 (최빈값),v2 (ISBN),해석
0,고유 언어 수,1,19,언어 종류가 많을수록 다양함
1,Shannon Entropy,-0.0000,0.9447,높을수록 다양함 (max: log2(N))
2,Gini-Simpson Index,0.0000,0.2881,높을수록 다양함 (0~1)



해석:
  ✅ v2가 v1보다 0.9447 더 다양함 (Entropy)
  ✅ v2가 v1보다 28.81% 더 다양함 (Gini-Simpson)

📋 최종 결론


,항목,v1,v2
0,처리 방법,최빈값 대체,ISBN 기반 추론
1,총 처리 건수,"67,227건","67,227건"
2,고유 언어 수,1개,19개
3,최다 언어,en,en
4,최다 언어 비율,100.0%,83.8%
5,Shannon Entropy,-0.0000,0.9447
6,Gini-Simpson Index,0.0000,0.2881
7,통계적 차이,-,p=0.00e+00



결론:
  ✅ v1과 v2의 언어 분포는 통계적으로 매우 유의미하게 다릅니다!
  ✅ ISBN 기반 추론이 더 다양한 언어를 올바르게 식별했습니다.
  → v2 (ISBN 기반) 추론이 더 정확하고 의미있는 결과입니다.

분석 완료!


# language feature의 중요도 분석

In [ ]:
"""
모델 학습 없이 Language Feature의 중요도 추정
"""

import pandas as pd
import numpy as np
from scipy import stats

print("=" * 80)
print("Language Feature 중요도 분석 (모델 학습 없이)")
print("=" * 80)

# 1. 데이터 로드
print("\n[Step 1] 데이터 로드")
train = pd.read_csv("/data/ephemeral/home/sojin/data/v4/train_ratings.csv")
books = pd.read_csv("/data/ephemeral/home/sojin/data/v4/books.csv")

# 병합
train_merged = train.merge(
    books[["isbn", "language", "category", "publisher", "book_author"]],
    on="isbn",
    how="left",
)

print(f"Train 데이터: {len(train_merged):,}건")
print(f"평균 평점: {train_merged['rating'].mean():.3f}")
print(f"평점 표준편차: {train_merged['rating'].std():.3f}")

# 2. Language별 평균 평점
print("\n" + "=" * 80)
print("📊 Language별 평균 평점")
print("=" * 80)

lang_stats = (
    train_merged.groupby("language")["rating"].agg(["mean", "count", "std"]).round(3)
)
lang_stats = lang_stats.sort_values("count", ascending=False)
lang_stats["mean_diff"] = (lang_stats["mean"] - train_merged["rating"].mean()).round(3)

display(lang_stats.head(20))

# 3. Category별 평균 평점 (비교용)
print("\n" + "=" * 80)
print("📊 Category별 평균 평점 (상위 20개)")
print("=" * 80)

cat_stats = (
    train_merged.groupby("category")["rating"].agg(["mean", "count", "std"]).round(3)
)
cat_stats = cat_stats.sort_values("count", ascending=False)
cat_stats["mean_diff"] = (cat_stats["mean"] - train_merged["rating"].mean()).round(3)

display(cat_stats.head(20))

# 4. 분산 분석
print("\n" + "=" * 80)
print("📈 분산 분석 (Variance Analysis)")
print("=" * 80)

total_var = train_merged["rating"].var()
lang_between_var = lang_stats["mean"].var()  # Language 간 평균의 분산
cat_between_var = cat_stats["mean"].var()  # Category 간 평균의 분산

print(f"전체 평점 분산: {total_var:.6f}")
print(f"\nLanguage 간 평균 분산: {lang_between_var:.6f}")
print(f"  → 전체 분산의 {lang_between_var/total_var*100:.4f}%")
print(f"\nCategory 간 평균 분산: {cat_between_var:.6f}")
print(f"  → 전체 분산의 {cat_between_var/total_var*100:.4f}%")

print(
    f"\n📊 Category가 Language보다 {cat_between_var/lang_between_var:.1f}배 더 영향력 큼!"
)

# 5. ANOVA (분산분석)
print("\n" + "=" * 80)
print("📈 ANOVA Test (통계적 유의성)")
print("=" * 80)

# Language ANOVA
language_groups = [
    group["rating"].values
    for name, group in train_merged.groupby("language")
    if len(group) >= 30
]
f_stat_lang, p_value_lang = stats.f_oneway(*language_groups)

print("Language ANOVA:")
print(f"  F-statistic: {f_stat_lang:.4f}")
print(f"  p-value: {p_value_lang:.2e}")
print(f"  해석: ", end="")
if p_value_lang < 0.001:
    print("Language가 Rating에 유의미한 영향을 줌 (p < 0.001)")
else:
    print("Language가 Rating에 유의미한 영향 없음")

# Category ANOVA
category_groups = [
    group["rating"].values
    for name, group in train_merged.groupby("category")
    if len(group) >= 30
]
f_stat_cat, p_value_cat = stats.f_oneway(*category_groups)

print("\nCategory ANOVA:")
print(f"  F-statistic: {f_stat_cat:.4f}")
print(f"  p-value: {p_value_cat:.2e}")
print(f"  해석: ", end="")
if p_value_cat < 0.001:
    print("Category가 Rating에 유의미한 영향을 줌 (p < 0.001)")
else:
    print("Category가 Rating에 유의미한 영향 없음")

# Publisher, Author도 비교
publisher_groups = [
    group["rating"].values
    for name, group in train_merged.groupby("publisher")
    if len(group) >= 30
]
f_stat_pub, p_value_pub = stats.f_oneway(*publisher_groups)

author_groups = [
    group["rating"].values
    for name, group in train_merged.groupby("book_author")
    if len(group) >= 30
]
f_stat_auth, p_value_auth = stats.f_oneway(*author_groups)

print("\nPublisher ANOVA:")
print(f"  F-statistic: {f_stat_pub:.4f}")

print("\nAuthor ANOVA:")
print(f"  F-statistic: {f_stat_auth:.4f}")

# 6. Feature 중요도 비교
print("\n" + "=" * 80)
print("🏆 Feature 중요도 순위 (F-statistic 기준)")
print("=" * 80)

importance_df = pd.DataFrame(
    {
        "Feature": ["Category", "Author", "Publisher", "Language"],
        "F-statistic": [f_stat_cat, f_stat_auth, f_stat_pub, f_stat_lang],
        "p-value": [p_value_cat, p_value_auth, p_value_pub, p_value_lang],
    }
)
importance_df = importance_df.sort_values("F-statistic", ascending=False)
importance_df["Relative_Importance"] = (
    importance_df["F-statistic"] / importance_df["F-statistic"].max() * 100
).round(1)

display(importance_df)

# 7. Language 평점 범위 분석
print("\n" + "=" * 80)
print("📊 평점 범위 분석")
print("=" * 80)

print(f"Language별 평균 평점 범위:")
print(f"  최소: {lang_stats['mean'].min():.3f}")
print(f"  최대: {lang_stats['mean'].max():.3f}")
print(f"  범위: {lang_stats['mean'].max() - lang_stats['mean'].min():.3f}")

print(f"\nCategory별 평균 평점 범위:")
print(f"  최소: {cat_stats['mean'].min():.3f}")
print(f"  최대: {cat_stats['mean'].max():.3f}")
print(f"  범위: {cat_stats['mean'].max() - cat_stats['mean'].min():.3f}")

# 8. 최종 결론
print("\n" + "=" * 80)
print("📋 최종 결론")
print("=" * 80)

lang_importance_pct = f_stat_lang / f_stat_cat * 100
summary = pd.DataFrame(
    {
        "항목": [
            "Language 분산 기여도",
            "Category 분산 기여도",
            "Language F-statistic",
            "Category F-statistic",
            "Language 상대 중요도",
            "평점 범위 (Language)",
            "평점 범위 (Category)",
        ],
        "값": [
            f"{lang_between_var/total_var*100:.4f}%",
            f"{cat_between_var/total_var*100:.4f}%",
            f"{f_stat_lang:.2f}",
            f"{f_stat_cat:.2f}",
            f"{lang_importance_pct:.1f}% (Category 대비)",
            f"{lang_stats['mean'].max() - lang_stats['mean'].min():.3f}",
            f"{cat_stats['mean'].max() - cat_stats['mean'].min():.3f}",
        ],
    }
)

display(summary)

print("\n해석:")
if lang_importance_pct < 10:
    print(f"  ❌ Language는 Category의 {lang_importance_pct:.1f}%만큼만 중요함")
    print(f"  ❌ Language가 평점 예측에 거의 기여하지 못함")
    print(f"  → Language를 추가해도 모델 성능 향상이 거의 없을 것으로 예상됨")
elif lang_importance_pct < 30:
    print(f"  ⚠️  Language는 Category의 {lang_importance_pct:.1f}% 정도 중요함")
    print(f"  → Language 추가 시 약간의 성능 향상 가능")
else:
    print(f"  ✅ Language는 Category의 {lang_importance_pct:.1f}% 만큼 중요함")
    print(f"  → Language 추가 시 성능 향상 기대 가능")

print("\n" + "=" * 80)
print("분석 완료!")
print("=" * 80)